# This notebook is purely meant for testing things out

In [1]:
import inspect
import os
from time import sleep

import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchmetrics
from sklearn.model_selection import KFold
from torchvision.transforms import transforms
from tqdm import tqdm

from src.s00_utils import configs, constants
from src.s01_data import datasets
from src.s03_modelling import models
from src.s04_training import runners, train

In [2]:
config = train.create_config()

In [3]:
config

Config(runner=CustomKFoldRunner(
  num_folds: 5
  num_epochs: 20
  batch_size: 32
  device: cpu
), data=CustomImageDataset(
  image_dir: C:\Users\E8J0G0K\Documents\Repos\ml-template\data\d01_raw\mnist\train
  label_path: C:\Users\E8J0G0K\Documents\Repos\ml-template\data\d01_raw\mnist\train\labels.csv
  transform: Compose(
    ToTensor()
    Grayscale(num_output_channels=1)
    Normalize(mean=0.5, std=0.2)
  )
), model=CustomConvNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
), optimizer=SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.01
    maximize: False
    momentum: 0.001
    nesterov: False
    weight_decay: 0
), loss_function=CrossEntropyLoss(), accuracy_function=Accuracy())

In [6]:
f"hi {config.runner}"

'hi CustomKFoldRunner(\n  num_folds: 5\n  num_epochs: 20\n  batch_size: 32\n  device: cpu\n)'

In [3]:
from torchvision import transforms

t = transforms.Compose([
        transforms.ToTensor(),
        transforms.Grayscale(),
        transforms.Normalize(0.5, 0.2),
    ])

In [4]:
t

Compose(
    ToTensor()
    Grayscale(num_output_channels=1)
    Normalize(mean=0.5, std=0.2)
)

In [76]:
from torch import nn
import torch.nn.functional as F


class CustomConvNet(nn.Module):
    def __init__(self, weight_init, activation):
        super(CustomConvNet, self).__init__()
        # Configs
        self.weight_init = weight_init
        self.activation = activation
        # Layers
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, 1)

    def init_weights(self):
        self.apply(self.weight_init)

    def __repr__(self):
        return super(CustomConvNet, self).__repr__() +"\n" \
                                                      f"{inspect.getsource(self.weight_init)}\n" \
                                                      f"Activation: {self.activation}"

In [77]:
def xavier_weight_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

In [78]:
model = CustomConvNet(xavier_weight_init, F.relu)

In [79]:
model

CustomConvNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)
def xavier_weight_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

Activation: <function relu at 0x000001EDDACAD9D0>

In [82]:
F.relu.

'<function relu at 0x000001EDDACAD9D0>'

In [83]:
class CustomWeightInit:
    def __init__(self, conv_init, bias):
        self.conv_init = conv_init
        self.bias =

    def weight_init(self, m):
        if isinstance(m, nn.Conv2d):
            self.conv_init(m.weight)

    def __repr__(self):
        return inspect.getsource(self.weight_init)

In [65]:
weight_init = CustomWeightInit(xavier_weight_init)

In [66]:
weight_init

def xavier_weight_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

In [67]:
weight_init.apply(model)

C:\Users\E8J0G0K\AppData\Local\Temp\ipykernel_4060\2205828600.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight)


In [2]:
models.CustomConvNetSequential()

CustomConvNetSequential(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (layers): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 5, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=2880, out_features=64, bias=True)
    (6): ReLU()
    (7): Linear(in_features=64, out_features=32, bias=True)
    (8): ReLU()
    (9): Linear(in_features=32, out_features=10, bias=True)
  )
)

In [89]:
config.data[0][0]

tensor([[[-2.5000, -2.5000, -2.5000, -2.5000, -2.5000, -2.5000, -2.5000,
          -2.5000, -2.3824, -2.5000, -2.2843, -2.5000, -2.2647, -2.4804,
          -2.3432, -2.3824, -2.4804, -2.4216, -2.4608, -2.4804, -2.5000,
          -2.4216, -2.2843, -2.4412, -2.5000, -2.5000, -2.5000, -2.5000],
         [-2.5000, -2.5000, -2.5000, -2.5000, -2.5000, -2.5000, -2.5000,
          -2.5000, -2.4804, -2.5000, -2.3432, -2.5000, -2.4412, -2.4608,
          -2.4804, -2.4216, -2.5000, -2.4412, -2.5000, -2.4804, -2.2843,
          -2.4216, -2.5000, -2.4804, -2.5000, -2.5000, -2.5000, -2.5000],
         [-2.5000, -2.5000, -2.5000, -2.5000, -2.5000, -2.5000, -2.5000,
          -2.5000, -2.5000, -2.4020, -2.5000, -2.4412, -2.4216, -2.5000,
          -2.4608, -2.4412, -2.2059, -2.4412, -2.5000, -2.3824, -2.2255,
          -2.4804, -2.4804, -2.0491, -2.5000, -2.5000, -2.5000, -2.5000],
         [-2.5000, -2.5000, -2.5000, -2.5000, -2.5000, -2.5000, -2.5000,
          -2.5000, -2.4020, -2.3628, -2.3235, -2

In [8]:
from torch import nn

def init_weights(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

In [9]:
import torch
torch.save(init_weights, "test.pt")

In [11]:
a = torch.load("test.pt")

In [13]:
from src.s03_modelling import models

net = models.SimpleNet()

In [14]:
net.apply(a)

C:\Users\E8J0G0K\AppData\Local\Temp\ipykernel_9020\699881287.py:5: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


SimpleNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [19]:
torch.cuda.device_count()

0

In [4]:
str(t)

'Compose(\n    ToTensor()\n    Grayscale(num_output_channels=1)\n    Normalize(mean=0.5, std=0.2)\n)'

In [5]:
import torch

torch.save(t, "test.pt")

In [6]:
p = torch.load("test.pt")

In [7]:
type(p)

torchvision.transforms.transforms.Compose

In [8]:
p

Compose(
    ToTensor()
    Grayscale(num_output_channels=1)
    Normalize(mean=0.5, std=0.2)
)

In [2]:
my_dict = {
    "Test": 4324343,
    "Hello": "waddup?"
}

In [13]:
str(my_dict)

"{'Test': 4324343, 'Hello': 'waddup?'}"

In [ ]:
def pretty(d, indent=0):
   for key, value in d.items():
      print('\t' * indent + str(key))
      if isinstance(value, dict):
         pretty(value, indent+1)
      else:
         print('\t' * (indent+1) + str(value))

In [20]:
pretty(my_dict)

Test
	4324343
Hello
	waddup?


In [1]:
from src.s00_utils import constants, configs
from src.s01_data import datasets
from src.s03_modelling import models
import os
import torch

output_dir = os.path.join(constants.ROOT, "models", "my_test")

# Initialize model
train_model = models.SimpleNet()

# Initialize data
train_data = datasets.MNIST_Dataset

# Initialize config
train_config = configs.Config(
    num_epochs=4,
    batch_size=32,
    learning_rate=0.01,
    momentum=0.5,
    num_folds=10,
    device=torch.device("cpu"),
    data=train_data,
    output_dir=output_dir,
)

In [8]:
import dataclasses

def as_tb_text(config):
    # Transform the config to a dictionary
    config_dict = dataclasses.asdict(config)

    for key, value in config_dict.items():
        if key == "data":
            pass
        elif key == "device"


    return

In [9]:
as_tb_text(train_config)

num_epochs 4
batch_size 32
learning_rate 0.01
momentum 0.5
num_folds 10
device cpu
data <src.s01_data.datasets.CustomImageDataset object at 0x0000023EB1836550>
output_dir C:\Users\E8J0G0K\Documents\Repos\ml-template\models\my_test


In [2]:
train_config.as_dict()

{'num_epochs': 4,
 'batch_size': 32,
 'learning_rate': 0.01,
 'momentum': 0.5,
 'num_folds': 10,
 'device': device(type='cpu'),
 'data': <src.s01_data.datasets.CustomImageDataset at 0x23eb1088190>,
 'output_dir': 'C:\\Users\\E8J0G0K\\Documents\\Repos\\ml-template\\models\\my_test'}

In [3]:
train_config.label_path

'C:\\Users\\E8J0G0K\\Documents\\Repos\\ml-template\\data\\mnist\\train\\labels.csv'

In [27]:
def pretty_config(d, indent=0):
   for key, value in d.items():
      print('\t' * indent + str(key))
      if isinstance(value, dict):
         pretty(value, indent+1)
      else:
         print('\t' * (indent+1) + str(value))

In [28]:
import dataclasses

pretty_config(dataclasses.asdict(train_config))

num_epochs
	4
batch_size
	32
learning_rate
	0.01
momentum
	0.5
num_folds
	10
device
	cpu
data
output_dir
	C:\Users\E8J0G0K\Documents\Repos\ml-template\models\my_test


In [29]:
train_config.as_dict()

'{"num_epochs": 4, "batch_size": 32, "learning_rate": 0.01, "momentum": 0.5, "num_folds": 10, "device": "cpu", "data": ["C:\\\\Users\\\\E8J0G0K\\\\Documents\\\\Repos\\\\ml-template\\\\data\\\\mnist\\\\train", "C:\\\\Users\\\\E8J0G0K\\\\Documents\\\\Repos\\\\ml-template\\\\data\\\\mnist\\\\train\\\\labels.csv", "Compose(\\n    ToTensor()\\n    Grayscale(num_output_channels=1)\\n    Normalize(mean=0.5, std=0.2)\\n)"], "output_dir": "C:\\\\Users\\\\E8J0G0K\\\\Documents\\\\Repos\\\\ml-template\\\\models\\\\my_test"}'

In [42]:
from src.s00_utils.configs import CustomJsonEncoder
import json
json.dumps(dataclasses.asdict(train_config), cls=CustomJsonEncoder, indent=2).replace("\\n", "\n").replace("\n", "  \n")

'{  \n  "num_epochs": 4,  \n  "batch_size": 32,  \n  "learning_rate": 0.01,  \n  "momentum": 0.5,  \n  "num_folds": 10,  \n  "device": "cpu",  \n  "data": [  \n    "C:\\\\Users\\\\E8J0G0K\\\\Documents\\\\Repos\\\\ml-template\\\\data\\\\mnist\\\\train",  \n    "C:\\\\Users\\\\E8J0G0K\\\\Documents\\\\Repos\\\\ml-template\\\\data\\\\mnist\\\\train\\\\labels.csv",  \n    "Compose(  \n    ToTensor()  \n    Grayscale(num_output_channels=1)  \n    Normalize(mean=0.5, std=0.2)  \n)"  \n  ],  \n  "output_dir": "C:\\\\Users\\\\E8J0G0K\\\\Documents\\\\Repos\\\\ml-template\\\\models\\\\my_test"  \n}'

In [12]:
import json
from torch.utils.tensorboard import SummaryWriter

summary_dir = r"C:\Users\E8J0G0K\Documents\Repos\ml-template\models\my_test\summary"
summary_writer = SummaryWriter(summary_dir)
summary_writer.add_text(tag="Experiment", text_string=str(json.dumps(my_dict, indent=2)).replace("\n", "  \n"))
summary_writer.close()

In [11]:
str(json.dumps(my_dict, indent=2)).replace("\n", "  \n")

'{  \n  "Test": 4324343,  \n  "Hello": "waddup?"  \n}'

*   Rot
*   Grün
*   Blau

**Learning rate:**

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned | $1600 |
| col 2 is      | centered      |   $12 |
| zebra stripes | are neat      |    $1 |

In [12]:
test = 3
result = f"| Param{test}eter | Value |  \n" \
             "| --------- | ----- |  \n" \
             "| "

In [60]:
isinstance(optimizer, torch.optim.Optimizer)

True

In [47]:
import torch
model = models.SimpleNet()
optimizer = torch.optim.SGD(model.parameters(), )
optimizer

SyntaxError: invalid syntax (2260141108.py, line 3)

In [57]:
optimizer

SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.001
    nesterov: False
    weight_decay: 0
)

In [50]:
model

SimpleNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [29]:
loss = torch.nn.CrossEntropyLoss

In [56]:
loss()

CrossEntropyLoss()

In [42]:
loss.c

torch.nn.modules.loss.CrossEntropyLoss

In [43]:
str(optimizer)

"<class 'torch.optim.sgd.SGD'>"

In [27]:
isinstance(optimizer, torch.optim.Optimizer)

False

In [34]:
isinstance(loss, type)

True

In [35]:
callable(loss)

True

In [33]:
loss.__class__

type

In [9]:
from src.s00_utils import configs

train_data = datasets.MNIST_Dataset

# Initialize config
train_config = configs.Config(
    num_epochs=4,
    batch_size=1024,
    learning_rate=0.01,
    momentum=0.5,
    num_folds=2,
    data=train_data
)

In [10]:
train_config.as_dict()

{'num_epochs': 4,
 'batch_size': 1024,
 'learning_rate': 0.01,
 'momentum': 0.5,
 'num_folds': 2,
 'data': <src.s01_data.datasets.CustomImageDataset at 0x26f2275e820>,
 'output_dir': 'D:\\GitRepos\\MyGitHub\\ml-template\\models\\run_20221009-161209'}

In [11]:
# train_config.to_file()

In [14]:
new_config = configs.Config.from_file("D:\\GitRepos\\MyGitHub\\ml-template\\models\\run_20221009-160648\\config.pt")

In [16]:
new_config.data

In [17]:
str(None)

'None'

In [54]:
torch.device("cpu").type

'cpu'

In [42]:
import glob
import os
import cv2
import csv

In [47]:
output_dir = "../data/mnist/train/"
with open(os.path.join(output_dir, "labels.csv"), "w", newline="") as f:
    writer = csv.writer(f)
    for file in glob.glob(r"C:\Users\E8J0G0K\Downloads\archive\trainingSample\trainingSample\*\*.jpg"):
        label = os.path.basename(os.path.dirname(file))
        name = os.path.basename(file)
        output_path = os.path.join(output_dir, name)
        print(label, name, output_path)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        img = cv2.imread(file)
        # cv2.imwrite(output_path, img)
        writer.writerow([name, label])

0 img_1.jpg ../data/mnist/train/img_1.jpg
0 img_108.jpg ../data/mnist/train/img_108.jpg
0 img_110.jpg ../data/mnist/train/img_110.jpg
0 img_111.jpg ../data/mnist/train/img_111.jpg
0 img_114.jpg ../data/mnist/train/img_114.jpg
0 img_129.jpg ../data/mnist/train/img_129.jpg
0 img_141.jpg ../data/mnist/train/img_141.jpg
0 img_146.jpg ../data/mnist/train/img_146.jpg
0 img_149.jpg ../data/mnist/train/img_149.jpg
0 img_153.jpg ../data/mnist/train/img_153.jpg
0 img_157.jpg ../data/mnist/train/img_157.jpg
0 img_17.jpg ../data/mnist/train/img_17.jpg
0 img_183.jpg ../data/mnist/train/img_183.jpg
0 img_188.jpg ../data/mnist/train/img_188.jpg
0 img_193.jpg ../data/mnist/train/img_193.jpg
0 img_195.jpg ../data/mnist/train/img_195.jpg
0 img_196.jpg ../data/mnist/train/img_196.jpg
0 img_200.jpg ../data/mnist/train/img_200.jpg
0 img_201.jpg ../data/mnist/train/img_201.jpg
0 img_203.jpg ../data/mnist/train/img_203.jpg
0 img_204.jpg ../data/mnist/train/img_204.jpg
0 img_23.jpg ../data/mnist/train/img_23.

Have a look at:

In [ ]:
from ignite.engine import create_supervised_trainer, create_supervised_evaluator, Events
from ignite.metrics import Accuracy


def train_on_single_fold(*args, fold_index=0, **kwargs):

    train_loader, val_loader = get_dataflow(fold_index=fold_index, **kwargs)
    model, optimizer, criterion, lr_scheduler = initialize_model(**kwargs)
    metrics = {
         "accuracy": Accuracy(),
    }

    trainer = create_supervised_trainer(model, optimizer, criterion)

    @trainer.on(Events.EPOCH_COMPLETED)
    def update_lr_scheduler(_):
        lr_scheduler.step()

    evaluator = create_supervised_evaluator(model, metrics=metrics)

    @trainer.on(Events.ITERATION_COMPLETED(every=validate_every))
    def validate(trainer):
        evaluator.run(val_loader)
        metrics = evaluator.state.metrics
        print("After {} iterations, binary accuracy = {:.2f}"
              .format(trainer.state.iteration, metrics['accuracy']))

    trainer.run(train_loader, max_epochs=kwargs.get(max_epochs, 100))